In [1]:
import numpy as np
import tensorflow as tf
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import pandas as pd
import os

# Capsule Layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(name='capsule_kernel',
                                      shape=(input_shape[-1], self.num_capsules * self.dim_capsule),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs):
        u_hat = tf.linalg.matmul(inputs, self.kernel)
        u_hat = tf.reshape(u_hat, (-1, inputs.shape[1], self.num_capsules, self.dim_capsule))
        u_hat = tf.transpose(u_hat, perm=[0, 2, 1, 3])
        for i in range(self.routings):
            c = tf.nn.softmax(tf.reduce_sum(u_hat, axis=-1, keepdims=True), axis=2)
            outputs = tf.linalg.matmul(c, u_hat, transpose_a=True)
        return tf.sqrt(tf.reduce_sum(tf.square(outputs), axis=-1))

# Length Layer (for output probabilities)
class Length(layers.Layer):
    def call(self, inputs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1))

# Capsule Network Model
def create_capsule_model(input_shape, n_classes):
    inputs = layers.Input(shape=input_shape)

    # Conv Layer
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Capsule Layer
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Reshape((-1, 128))(x)
    capsules = CapsuleLayer(num_capsules=n_classes, dim_capsule=16, routings=3)(x)

    # Output Layer
    output = Length()(capsules)

    model = models.Model(inputs=inputs, outputs=output)
    return model

# Kaggle-specific setup
if __name__ == "__main__":
    # Path to HAM10000 dataset files
    metadata_path = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"
    images_dir1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/"
    images_dir2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/"

    # Load metadata
    metadata = pd.read_csv(metadata_path)

    # Combine image paths
    metadata['image_path'] = metadata['image_id'].apply(
        lambda x: images_dir1 + x + ".jpg" if os.path.exists(images_dir1 + x + ".jpg") else images_dir2 + x + ".jpg"
    )

    # Encode labels
    le = LabelEncoder()
    metadata['label'] = le.fit_transform(metadata['dx'])

    # Load images and preprocess
    images = []
    labels = []

    for _, row in metadata.iterrows():
        img = load_img(row['image_path'], target_size=(224, 224))  # Resize to match model input
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(row['label'])

    # Convert to NumPy arrays
    images = np.array(images)
    labels = to_categorical(labels, num_classes=7)  # One-hot encode labels

    # Data Augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Shuffle and split the data
    images, labels = shuffle(images, labels, random_state=42)
    train_data, val_data, train_labels, val_labels = train_test_split(
        images, labels, test_size=0.2, random_state=42
    )

    # Create model
    input_shape = (224, 224, 3)  # Adjust for HAM10000 image sizes
    n_classes = 7  # Number of skin cancer types
    model = create_capsule_model(input_shape, n_classes)

    # Compile the model
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(
        datagen.flow(train_data, train_labels, batch_size=32),
        validation_data=(val_data, val_labels),
        epochs=50,  # Increased epochs for better performance
        steps_per_epoch=len(train_data) // 32
    )

    # Save the model
    model.save("capsule_model.h5")

    print("Model training completed and saved as capsule_model.h5")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 102s 348ms/step - accuracy: 0.6548 - loss: 1.2946 - val_accuracy: 0.6840 - val_loss: 1.0714
Epoch 2/50
  1/250 ━━━━━━━━━━━━━━━━━━━━ 26s 107ms/step - accuracy: 0.6250 - loss: 1.1390

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6250 - loss: 1.1390 - val_accuracy: 0.6840 - val_loss: 1.0742
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 290ms/step - accuracy: 0.6705 - loss: 1.0604 - val_accuracy: 0.6840 - val_loss: 1.0010
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8125 - loss: 0.7592 - val_accuracy: 0.6840 - val_loss: 0.9979
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 287ms/step - accuracy: 0.6600 - loss: 1.0384 - val_accuracy: 0.6845 - val_loss: 0.9831
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5938 - loss: 1.2846 - val_accuracy: 0.6840 - val_loss: 0.9921
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 291ms/step - accuracy: 0.6641 - loss: 0.9950 - val_accuracy: 0.6835 - val_loss: 0.9265
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6875 - loss: 0.9771 - val_accuracy: 0.6835 - val_loss: 0.9284
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 291ms/step - accuracy: 0.6733 - loss: 0.9362 - val_accuracy: 0

In [2]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import gradio as gr
from keras.models import load_model
from PIL import Image
import numpy as np
import keras.backend as K
from tensorflow.keras.layers import Layer

# Define CapsuleLayer class with a default value for dim_capsules
class CapsuleLayer(Layer):
    def __init__(self, num_capsules, dim_capsules=16, routings=3, **kwargs):
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules  # Default value for dim_capsules
        self.routings = routings
        super(CapsuleLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Weight initialization
        self.W = self.add_weight(name="capsule_weight",
                                 shape=(input_shape[1], self.num_capsules, self.dim_capsules),
                                 initializer="glorot_uniform",
                                 trainable=True)
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        # Capsule forward pass (simplified)
        u_hat = K.batch_dot(inputs, self.W, [2, 1])  # Apply the transformation matrix
        return u_hat

# Load the Capsule Network model
try:
    model = load_model("capsule_model.h5", custom_objects={"CapsuleLayer": CapsuleLayer}, compile=False)
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")

# Define class labels
class_labels = [
    'Melanocytic nevi',
    'Melanoma',
    'Benign keratosis-like lesions',
    'Basal cell carcinoma',
    'Actinic keratoses',
    'Vascular lesions',
    'Dermatofibroma'
]

# Image preprocessing function
def preprocess_image(image):
    image = image.resize((224, 224))  # Resize to the correct model's expected input size
    image = np.array(image)  # Convert image to numpy array
    image = image.astype('float32')  # Convert to float32
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Normalize the image
    return image

# Prediction function
def predict_skin_cancer(image):
    # Preprocess the image
    image = preprocess_image(image)

    # Predict the class probabilities
    predictions = model.predict(image)

    # Get the class with the highest probability
    predicted_class = np.argmax(predictions, axis=1)

    # Get the corresponding label
    predicted_label = class_labels[predicted_class[0]]

    # Get the confidence score (probability of the predicted class)
    confidence_score = np.max(predictions) * 100

    return predicted_label, confidence_score

# Custom CSS for title box with frame and larger text size (no purple theme)
css = """
h1 {
    font-size: 40px;
    text-align: center;
    padding: 20px;
    background-color: #e55b13;
    color: white;
    border: 2px solid #333;
    border-radius: 10px;
}

body {
    background-color: #f0f0f0;
    color: black;
    font-family: Arial, sans-serif;
}

.gradio-container {
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    padding: 20px;
}


"""

# Create Gradio interface
interface = gr.Interface(
    fn=predict_skin_cancer,
    inputs=gr.Image(type="pil", label="Upload a Dermoscopic Image", interactive=True),
    outputs=[
        gr.Textbox(label="Predicted Skin Cancer Type"),
        gr.Textbox(label="Confidence Score (%)")
    ],
    title="Skin Cancer Detection System",
    description="This system uses a Capsule Network to detect 7 types of skin cancer from dermoscopic images.",
    theme=None,  # No theme applied
    allow_flagging="never",  # Optional: hide flagging
    css=css  # Apply custom CSS
)

# Launch the interface
interface.launch(share=True)


Error loading model: Error when deserializing class 'CapsuleLayer' using config={'name': 'capsule_layer', 'num_capsules': 7, 'dim_capsule': 16, 'routings': 3, 'trainable': True, 'dtype': 'float32'}.

Exception encountered: Unrecognized keyword arguments passed to CapsuleLayer: {'dim_capsule': 16}


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://240eb555f02edccb38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
import gradio as gr
from keras.models import load_model
from PIL import Image
import numpy as np
import keras.backend as K
from tensorflow.keras.layers import Layer

# Define CapsuleLayer class with a default value for dim_capsules
class CapsuleLayer(Layer):
    def __init__(self, num_capsules, dim_capsules=16, routings=3, **kwargs):
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules  # Default value for dim_capsules
        self.routings = routings
        super(CapsuleLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Weight initialization
        self.W = self.add_weight(name="capsule_weight",
                                 shape=(input_shape[1], self.num_capsules, self.dim_capsules),
                                 initializer="glorot_uniform",
                                 trainable=True)
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        # Capsule forward pass (simplified)
        u_hat = K.batch_dot(inputs, self.W, [2, 1])  # Apply the transformation matrix
        return u_hat

# Load the Capsule Network model
try:
    model = load_model("capsule_model.h5", custom_objects={"CapsuleLayer": CapsuleLayer}, compile=False)
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")

# Define class labels
class_labels = [
    'Melanocytic nevi',
    'Melanoma',
    'Benign keratosis-like lesions',
    'Basal cell carcinoma',
    'Actinic keratoses',
    'Vascular lesions',
    'Dermatofibroma'
]

# Image preprocessing function
def preprocess_image(image):
    image = image.resize((224, 224))  # Resize to the correct model's expected input size
    image = np.array(image)  # Convert image to numpy array
    image = image.astype('float32')  # Convert to float32
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Normalize the image
    return image

# Prediction function
def predict_skin_cancer(image):
    # Preprocess the image
    image = preprocess_image(image)

    # Predict the class probabilities
    predictions = model.predict(image)

    # Get the class with the highest probability
    predicted_class = np.argmax(predictions, axis=1)

    # Get the corresponding label
    predicted_label = class_labels[predicted_class[0]]

    # Get the confidence score (probability of the predicted class)
    confidence_score = np.max(predictions) * 100

    return predicted_label, confidence_score

# Custom CSS for purple theme and title box with frame
css = """
body {
    background-color: #7A4E96;
    color: white;
    font-family: Arial, sans-serif;
}

h1 {
    font-size: 36px;
    text-align: center;
    padding: 20px;
    background-color: #5d3377;
    border-radius: 10px;
    border: 2px solid #7A4E96;
}

h3 {
    font-size: 28px;
}

.gradio-container {
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    padding: 20px;
}



button {
    background-color: #7A4E96;
    color: white;
    font-size: 18px;
    border: none;
    padding: 10px 20px;
    border-radius: 5px;
    cursor: pointer;
}

button:hover {
    background-color: #5d3377;
}


"""

# Create Gradio interface
interface = gr.Interface(
    fn=predict_skin_cancer,
    inputs=gr.Image(type="pil", label="Upload a Dermoscopic Image", interactive=True),
    outputs=[
        gr.Textbox(label="Predicted Skin Cancer Type"),
        gr.Textbox(label="Confidence Score (%)")
    ],
    title="Skin Cancer Detection System",
    description="This system uses a Capsule Network to detect 7 types of skin cancer from dermoscopic images.",
    theme="default",  # Use the default theme
    allow_flagging="never",  # Optional: hide flagging
    css=css  # Apply custom CSS
)

# Launch the interface
interface.launch(share=True)


Error loading model: Error when deserializing class 'CapsuleLayer' using config={'name': 'capsule_layer', 'num_capsules': 7, 'dim_capsule': 16, 'routings': 3, 'trainable': True, 'dtype': 'float32'}.

Exception encountered: Unrecognized keyword arguments passed to CapsuleLayer: {'dim_capsule': 16}


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://668a6b7a4f4fe02b59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
!pip install huggingface_hub